In [ ]:
import os
import sys

# necessary for calling stuff since we are in notebook folder
# Add parent directory to path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
os.chdir(parent_dir)

from dotenv import load_dotenv
from openai import OpenAI
import json
from services import UseCaseService


In [ ]:
# load API key
load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

In [ ]:
# init client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key
)

service = UseCaseService()

# Tools

In [ ]:
# Tool 1: Get all use cases
tool_get_all_use_cases = {
    "type": "function",
    "function": {
        "name": "get_all_use_cases",
        "description": (
            "Retrieve all use cases from the database. "
            "Use this when the user wants to see all use cases, list use cases, "
            "or get an overview of everything in the system. "
            "Returns comprehensive information including title, description, status, "
            "company, and industry for each use case."
        ),
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    }
}

# Tool 2: Get use case by ID
tool_get_use_case_by_id = {
    "type": "function",
    "function": {
        "name": "get_use_case_by_id",
        "description": (
            "Get detailed information about a specific use case by its ID number. "
            "Use this when the user mentions a specific use case number or ID "
            "(e.g., 'use case 5', 'UC #3', 'number 7'). "
            "Returns all details including title, description, status, company, industry, and benefits."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "use_case_id": {
                    "type": "integer",
                    "description": "The numeric ID of the use case to retrieve (e.g., 1, 2, 3, etc.)"
                }
            },
            "required": ["use_case_id"]
        }
    }
}

# Tool 3: Create use case
tool_create_use_case = {
    "type": "function",
    "function": {
        "name": "create_use_case",
        "description": (
            "Create a new use case in the database. "
            "Use this when the user wants to add a new use case, extract use cases from transcripts, "
            "or create entries based on workshop discussions. "
            "IMPORTANT: You must provide company_id and industry_id as integers. "
            "If the user mentions company/industry names, you may need to ask for clarification "
            "or reference existing data to find the correct IDs."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string",
                    "description": "The title/name of the use case (required, must not be empty). Be concise but descriptive."
                },
                "description": {
                    "type": "string",
                    "description": (
                        "Detailed description of what the use case involves, the problem it solves, "
                        "and how it works (optional but recommended)"
                    )
                },
                "expected_benefit": {
                    "type": "string",
                    "description": (
                        "Expected benefits, value, or outcomes of implementing this use case "
                        "(optional). Include quantitative metrics if available."
                    )
                },
                "company_id": {
                    "type": "integer",
                    "description": (
                        "ID of the company this use case belongs to (required). "
                        "This must be a valid company ID number from the database."
                    )
                },
                "industry_id": {
                    "type": "integer",
                    "description": (
                        "ID of the industry this use case belongs to (required). "
                        "This must be a valid industry ID number from the database."
                    )
                },
                "status": {
                    "type": "string",
                    "description": (
                        "Initial status for the use case (optional, defaults to 'new'). "
                        "Must be EXACTLY one of the valid status values listed in enum. "
                        "Map user's language/intent to these exact values: "
                        "'neu'/'new'→'new', 'in Prüfung'/'in review'→'in_review', "
                        "'genehmigt'/'approved'→'approved', 'in Arbeit'/'in progress'→'in_progress', "
                        "'fertig'/'done'/'completed'→'completed', 'archiviert'/'archived'→'archived'"
                    ),
                    "enum": ["new", "in_review", "approved", "in_progress", "completed", "archived"]
                }
            },
            "required": ["title", "company_id", "industry_id"]
        }
    }
}

# Tool 4: Update use case
tool_update_use_case = {
    "type": "function",
    "function": {
        "name": "update_use_case",
        "description": (
            "Update an existing use case. Only the fields you provide will be changed; "
            "all other fields remain unchanged. "
            "Use this when the user wants to modify, change, or edit a use case. "
            "You can update any combination of title, description, benefit, status, company, or industry."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "use_case_id": {
                    "type": "integer",
                    "description": "ID of the use case to update (required)"
                },
                "title": {
                    "type": "string",
                    "description": "New title (optional). If provided, must not be empty."
                },
                "description": {
                    "type": "string",
                    "description": "New description (optional)"
                },
                "expected_benefit": {
                    "type": "string",
                    "description": "New expected benefit (optional)"
                },
                "status": {
                    "type": "string",
                    "description": (
                        "New status (optional). Must be EXACTLY one of the valid values in enum. "
                        "Translate user's intent: 'neu'→'new', 'In Bewertung'/'zur Prüfung'→'in_review', "
                        "'genehmigt'/'freigegeben'→'approved', 'in Bearbeitung'/'in Arbeit'→'in_progress', "
                        "'abgeschlossen'/'fertig'→'completed', 'archiviert'→'archived'"
                    ),
                    "enum": ["new", "in_review", "approved", "in_progress", "completed", "archived"]
                },
                "company_id": {
                    "type": "integer",
                    "description": "New company ID (optional). Must be a valid company ID from the database."
                },
                "industry_id": {
                    "type": "integer",
                    "description": "New industry ID (optional). Must be a valid industry ID from the database."
                }
            },
            "required": ["use_case_id"]
        }
    }
}

# Tool 5: Update status (specialized)
tool_update_use_case_status = {
    "type": "function",
    "function": {
        "name": "update_use_case_status",
        "description": (
            "Update ONLY the status of a use case. This is a convenience function for status-only changes. "
            "Use this when the user wants to change, set, or update just the status "
            "(e.g., 'approve use case 5', 'mark case 3 as completed', 'set UC 7 to in progress'). "
            "CRITICAL: Always map the user's language/intent to one of the exact valid status values."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "use_case_id": {
                    "type": "integer",
                    "description": "ID of the use case whose status should be changed"
                },
                "status": {
                    "type": "string",
                    "description": (
                        "New status value. Must be EXACTLY one of the values in enum. "
                        "Language mapping examples: "
                        "German: 'neu'→'new', 'In Bewertung'/'In Prüfung'→'in_review', 'genehmigt'→'approved', "
                        "'in Bearbeitung'/'läuft'→'in_progress', 'abgeschlossen'/'fertig'/'erledigt'→'completed', "
                        "'archiviert'→'archived'. "
                        "English variants: 'reviewing'→'in_review', 'working on'/'ongoing'→'in_progress', "
                        "'done'/'finished'→'completed'"
                    ),
                    "enum": ["new", "in_review", "approved", "in_progress", "completed", "archived"]
                }
            },
            "required": ["use_case_id", "status"]
        }
    }
}

# Tool 6: Delete use case
tool_delete_use_case = {
    "type": "function",
    "function": {
        "name": "delete_use_case",
        "description": (
            "Permanently delete a use case from the database. "
            "Use this when the user explicitly wants to remove, delete, or eliminate a use case. "
            "WARNING: This action cannot be undone. The function returns information about "
            "the deleted use case for confirmation. "
            "Consider suggesting archiving instead of deletion when appropriate."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "use_case_id": {
                    "type": "integer",
                    "description": "ID of the use case to delete permanently"
                }
            },
            "required": ["use_case_id"]
        }
    }
}

# Tool 7: Filter use cases
tool_filter_use_cases = {
    "type": "function",
    "function": {
        "name": "filter_use_cases",
        "description": (
            "Filter and search use cases by various criteria. All filters are optional - "
            "you can use one or combine multiple filters. "
            "Use this when the user wants use cases matching specific conditions "
            "(e.g., 'show energy sector use cases', 'what's in progress', 'cases from company X'). "
            "Returns a list of use cases matching ALL provided filters (AND logic)."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "industry_id": {
                    "type": "integer",
                    "description": (
                        "Filter by industry ID (optional). "
                        "Only return use cases belonging to this specific industry. "
                        "Must be a valid industry ID number from the database."
                    )
                },
                "company_id": {
                    "type": "integer",
                    "description": (
                        "Filter by company ID (optional). "
                        "Only return use cases belonging to this specific company. "
                        "Must be a valid company ID number from the database."
                    )
                },
                "status": {
                    "type": "string",
                    "description": (
                        "Filter by status (optional). Only return use cases with this exact status. "
                        "Must be EXACTLY one of the valid values in enum. "
                        "Map user's language: 'neu'/'new'→'new', 'in Bewertung'/'zur Prüfung'→'in_review', "
                        "'genehmigt'/'approved'→'approved', 'laufend'/'in Arbeit'/'in progress'→'in_progress', "
                        "'fertig'/'abgeschlossen'/'done'→'completed', 'archiviert'/'archived'→'archived'"
                    ),
                    "enum": ["new", "in_review", "approved", "in_progress", "completed", "archived"]
                },
                "person_id": {
                    "type": "integer",
                    "description": (
                        "Filter by person who contributed (optional). "
                        "Only return use cases that this specific person contributed to. "
                        "Must be a valid person ID number from the database."
                    )
                }
            },
            "required": []
        }
    }
}

# Combine all tools into a list
tools = [
    tool_get_all_use_cases,
    tool_get_use_case_by_id,
    tool_create_use_case,
    tool_update_use_case,
    tool_update_use_case_status,
    tool_delete_use_case,
    tool_filter_use_cases
]

# Tool -> function mapping

In [ ]:
tool_functions = {
    "get_all_use_cases": service.get_all_use_cases,
    "get_use_case_by_id": service.get_use_case_by_id,
    "create_use_case": service.create_use_case,
    "update_use_case": service.update_use_case,
    "update_use_case_status": service.update_use_case_status,
    "delete_use_case": service.delete_use_case,
    "filter_use_cases": service.filter_use_cases
}

# implement a function that takes the function name and calls the real function and returns result

In [ ]:
def execute_tool(function_name : str, arguments : dict):

    # check if function exists
    if function_name not in tool_functions.keys():
        return {"error" : f"Unknown function: {function_name}"}

    try: 
        actual_function = tool_functions[function_name]

        # call the function
        result = actual_function(**arguments)

        return result
    
    except Exception as e:
        return {"error" : str(e)}

In [ ]:
def run_agent(user_message : str, verbose: bool = True):
    print(f"\nUser: {user_message}\n")

    # (1) Call agent with user message
    agent_response_1 = client.chat.completions.create(
        model="anthropic/claude-3.5-sonnet",
        messages=[
            {"role" : "user", "content" : user_message}
        ],
        tools=tools, 
        max_tokens=2000
    )
    agent_message_1 = agent_response_1.choices[0].message

    # (2) the agent might want to call a tool or more
    if agent_message_1.tool_calls:
        if verbose:
            print(f"\nThe agent wants to call {len(agent_message_1.tool_calls)} tools.")


        # safe dialog
        all_messages = [
            {"role" : "user", "content" : user_message},  # user input
            agent_message_1  # agent's first response containing the tool call
        ]

        # execute the agent's tool call or all tool calls
        for tool_call in agent_message_1.tool_calls: 

            # get function name and it's arguments
            function_name = tool_call.function.name
            arguments_str = tool_call.function.arguments
            
            # format arguments if nestled in str
            if arguments_str and arguments_str.strip():
                arguments = json.loads(arguments_str)
            else:
                arguments = {}  # Empty dict for functions with no parameters

            if verbose:
                print(f"\nAgent calls {function_name}")
                if arguments:
                    print(f"Arguments: {arguments}")
                else:
                    print(f"Arguments: (none)")
            
            # runs the tool function
            result = execute_tool(function_name, arguments)

            if verbose:
                if isinstance(result, list):
                    print(f"Returned {len(result)} items")
                elif isinstance(result, dict):
                    if "error" in result:
                        print(f"Error: {result['error']}")
                    else:
                        print(f"Success: {result.get('title', 'OK')}")
                else:
                    print(f"Result: {result}")

            # add the result to the whole dialog
            all_messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })

        # (3) send to result and all messages back to the LLM for final answer, this time without any tools
        if verbose:
            print(f"\nSending results back to LLM for final answer...")
        agent_response_2 = client.chat.completions.create(
            model="anthropic/claude-3.5-sonnet",
            messages=all_messages,
            max_tokens=2000
        )
        final_answer = agent_response_2.choices[0].message.content
        print(f"Agent: {final_answer}")

        return final_answer
    
    else:  # agent does not want to call any tools
        if verbose:
            print(f"\nAgent responding directly (no tools needed)")
        print(f"Agent: {agent_message_1.content}")

        return agent_message_1.content

# Test some calls

In [ ]:
run_agent("Show me all use cases")

In [ ]:
run_agent("Tell me about use case number 1")

In [ ]:
run_agent("Please change the status of use case 12 to in_review")


In [ ]:
run_agent("please show me all use cases with status in_review. Please also share their IDs.")

# Test tool calls

In [ ]:
print("\n" + "="*80)
print("TEST 1: GET ALL USE CASES")
print("="*80)
run_agent("Show me all use cases")

# Test 2: Get specific use case
print("\n" + "="*80)
print("TEST 2: GET USE CASE BY ID")
print("="*80)
run_agent("Tell me about use case number 1")

# Test 3: Filter by status
print("\n" + "="*80)
print("TEST 3: FILTER BY STATUS")
print("="*80)
run_agent("Show me all use cases with status 'new'")


# Test 4: Update status
print("\n" + "="*80)
print("TEST 4: UPDATE STATUS")
print("="*80)
run_agent("Change use case 1 to status 'in_progress'")

# Test 5: Create use case
print("\n" + "="*80)
print("TEST 5: CREATE USE CASE")
print("="*80)
run_agent("Create a new use case called 'Test Agent Integration' for company 1 in industry 2")

In [ ]:
print("\n" + "="*80)
print("FURTHER AGENT TESTING")
print("="*80)

# Test 1: German status update
print("\n--- TEST 1: German Status Update ---")
run_agent("Bitte setze Use Case 1 auf Status 'In Bewertung'", verbose=True)

# Test 2: Typo tolerance
print("\n--- TEST 2: Typo in Status ---")
run_agent("Show me use cases that are 'in progres'", verbose=True)

# Test 3: Mixed language
print("\n--- TEST 3: Mixed German/English ---")
run_agent("Filter nach Status abgeschlossen", verbose=True)

# Test 4: Natural language status
print("\n--- TEST 4: Natural Language Status ---")
run_agent("Mark use case 2 as done", verbose=True)

# Test 5: Multiple variations
print("\n--- TEST 5: Status Variation ---")
run_agent("Set use case 3 to 'reviewing'", verbose=True)

# Test 6: Complex query
print("\n--- TEST 6: Filter Combination ---")
run_agent("Show me all use cases in industry 1 that are approved", verbose=True)

# Test 7: Create with German status
print("\n--- TEST 7: Create with German ---")
run_agent("Erstelle einen neuen Use Case 'Test Deutsch' für Firma 1 in Branche 1 mit Status 'genehmigt'", verbose=True)

In [ ]:
print("\n--- TEST 8:  ---")
run_agent("Please show me all IT related use cases", verbose=True)